<font size = "5">

Mean Reversion Trading Strategy

In [15]:
import pandas as pd
import numpy as np

<font size = "4">

Step 0: Collect our data from our priceData function. For this walkthrough we will use ``KO`` Coca-Cola as an example.

In [16]:
from quantlab.data.yf import priceData
from quantlab.data.tickers import tickers

data = priceData(tickers, 150)
s = pd.Series(data["KO"])

[*********************100%***********************]  239 of 239 completed

17 Failed downloads:
['SNOW', 'CP', 'KLAC', 'BEN', 'APO', 'BMY', 'TMO', 'LCID', 'NNN', 'TSLA', 'PATH', 'LVS', 'IRM', 'TM', 'GS', 'CAH']: Timeout('Failed to perform, curl: (28) Connection timed out after 10002 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['CNI']: Timeout('Failed to perform, curl: (28) Connection timed out after 10001 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


<font size = "4">

Step 1: For each stock, create arrays of the stock orderings ``t``, prices ``s``, and price logarithms ``y``. We use log to compare changes more accurately.

In [17]:
n = len(s)
t = np.arange(n)
y = np.log(s.values)

In [18]:
print(f"n: {n}")
print(f"t: {t[:10]} ...")
print(f"y: {y[:10]} ...")

n: 104
t: [0 1 2 3 4 5 6 7 8 9] ...
y: [4.26194138 4.25020614 4.25315291 4.24668675 4.25287255 4.25874181
 4.26679064 4.26291314 4.26526912 4.25532156] ...


<font size = "4">

Step 2: Calculate mean values of ``y`` and ``t`` to build a regression line of the price data over the past ``n`` days. Then, create a series of expected y values ``exp_ys`` and expected prices ``exp_ps`` to compare with our actual y values.

In [19]:
t_mean = t.mean()
y_mean = y.mean()
slope = np.sum((t - t_mean) * (y - y_mean)) / np.sum((t - t_mean)**2)
intercept = y_mean - slope * t_mean
exp_ys = intercept + slope * t
exp_ps = np.exp(exp_ys)

In [20]:
print(f"t_mean: {t_mean}")
print(f"y_mean: {y_mean}")
print(f"slope: {slope}")
print(f"intercept: {intercept}")
print(f"exp_ys: {exp_ys[:10]} ...")
print(f"exp_ps: {exp_ps[:10]} ...")

t_mean: 51.5
y_mean: 4.228343654116266
slope: -0.00044311990369876064
intercept: 4.251164329156752
exp_ys: [4.25116433 4.25072121 4.25027809 4.24983497 4.24939185 4.24894873
 4.24850561 4.24806249 4.24761937 4.24717625] ...
exp_ps: [70.18708566 70.15599126 70.12491063 70.09384377 70.06279067 70.03175133
 70.00072574 69.9697139  69.93871579 69.90773142] ...


<font size = "4">

Step 3: Compare our expectations with the actual prices and calculate ``error_threshold`` based on standard deviation of our errors. Then, compare ``current_price`` with ``expected_current_price`` to figure out the ``error``. If the error is significant enough, we believe the stock is overvalued/undervalued and we should make a move on it.

In [21]:
residuals = s.values - exp_ps
error_threshold = residuals.std()
current_price = s.iloc[-1]
exp_current_price = exp_ps[-1]
error = current_price - exp_current_price

In [22]:
print(f"residuals: {residuals[:10]} ...")
print(f"error_threshold: {error_threshold}")
print(f"current_price: {current_price}")
print(f"exp_current_price: {exp_current_price}")
print(f"error: {error}")

residuals: [ 0.7605004  -0.03612584  0.20188686 -0.22032387  0.24429246  0.68919594
  1.29173947  1.04684952  1.24535861  0.57174551] ...
error_threshold: 1.2129566984392965
current_price: 70.16000366210938
exp_current_price: 67.05565720343351
error: 3.104346458675863


<font size = "4">
Step 4: If the error is over 2 times the error threshold, meaning the stock overprices the mean by 2 standard deviations, we believe that the stock will revert back to its mean and therefore we short the stock now. If the error is less than -2 times the error threshold, the stock underprices the mean by 2 standard deviations, and we long the stock expecting it to grow and revert back to its mean. The amount we buy should be standardized by price, so we divide 100 by the price of a share in order to buy/sell $100 of each stock.

In [23]:
if error > 2 * error_threshold:
    result = -100 / current_price
    print("Since the result is greater than the mean minus 2 standard deviations, short the stock.")
elif error < -2 * error_threshold:
    result = 100 / current_price
    print("Since the result is less than the mean minus 2 standard deviations, long the stock.")
else:
    print("Since the error is within 2 standard deviations of the mean, do nothing.")

Since the result is greater than the mean minus 2 standard deviations, short the stock.


<font size = "4">
Now, Wrap all of this as a function, looping across all columns. Start by initializing an empty data frame ``results``, which will be added to if our result consists of us buying or selling a stock. Return the data frame of trading decisions of all stocks.

In [24]:
def meanReversion(data):
    results = {}
    for col in data.columns:
        s = pd.Series(data[col])
        n = len(s)
        t = np.arange(n)
        y = np.log(s.values)
        t_mean = t.mean()
        y_mean = y.mean()
        slope = np.sum((t - t_mean) * (y - y_mean)) / np.sum((t - t_mean)**2)
        intercept = y_mean - slope * t_mean
        exp_ys = intercept + slope * t
        exp_ps = np.exp(exp_ys)
        residuals = s.values - exp_ps
        error_threshold = residuals.std()
        current_price = s.iloc[-1]
        exp_current_price = exp_ps[-1]
        error = current_price - exp_current_price

        if error > 2 * error_threshold:
            results[col] = -100 / current_price
        elif error < -2 * error_threshold:
            results[col] = 100 / current_price
        else:
            continue
    return pd.Series(results, name = "shares")

In [25]:
results = meanReversion(data)
print(results)

AFG      0.764409
ALL      0.523972
AMD     -0.387582
ARE      1.588815
BLK      0.088893
BX       0.659239
CINF     0.658979
CRWD    -0.182835
CSCO    -1.377031
D        1.677290
DOW     -3.940110
EW      -1.216693
GM      -1.431229
HIG      0.815062
IBM     -0.319928
ISRG    -0.183355
JNJ      0.534960
KO      -1.425313
LIN      0.225876
MDT      1.072616
MKL      0.054408
MU      -0.450633
NCLH     4.474273
NIO     14.104372
NSC      0.356633
NVS      0.810176
OXY      2.449179
PGR      0.471898
QCOM    -0.552395
REGN    -0.152793
RMD      0.389757
SPG      0.565707
STLA    -8.841733
UAL      1.053297
UPS     -1.037775
VICI     3.293808
WPC      1.512630
Name: shares, dtype: float64
